In [1]:
# import numpy as np
# import pandas as pd
# from tqdm import tqdm
# import random
# import os
# import json

# from lof import LocalOutlierFactor
# from matrix_profile import MatrixProfile
# from isolation_forest import IsolationForest
# from kmeans import KMeans
# from ensemble_detector import EnsembleDetector
# from dataloader import DataLoader
# from benchmarker import benchmark

# # Constants
# UCR_PATH = 'ucrdata'
# CACHED_SCORES_DIR = 'scores'
# RESULTS_DIR = 'results'
# ENSEMBLE_RESULTS_DIR = 'ensembles/results'
# ENSEMBLE_SCORES_DIR = 'ensembles/scores'
# N_REPETITIONS = 30
# ENSEMBLE_SIZE = 20
# BASE_LEARNERS_PER_TYPE = 5

# # Base learner configurations
# BASE_LEARNERS = {
#     'LOF': {
#         'class': LocalOutlierFactor,
#         'params': [
#             {'windowSize': ws, 'neighbors': n, 'gpu': True}
#             for ws in [25, 50, 100, 150, 200, 250]
#             for n in [10, 20, 50, 100]
#         ]
#     },
#     'IF': {
#         'class': IsolationForest,
#         'params': [
#             {'windowSize': ws}
#             for ws in [20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 250, 300, 350, 400, 450, 500, 550, 600]
#         ]
#     },
#     'KMeans': {
#         'class': KMeans,
#         'params': [
#             {'windowSize': ws, 'n_clusters': nc}
#             for ws in [50, 100, 200, 500]
#             for nc in [10, 20, 50, 100, 200]
#         ]
#     },
#     'MP': {
#         'class': MatrixProfile,
#         'params': [
#             {'windowSize': ws}
#             for ws in [20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 250, 300, 350, 400, 450, 500, 550, 600]
#         ]
#     }
# }

# # Ensemble methods
# ENSEMBLE_METHODS = [
#     ('simple_average', {}),
#     ('maximum_score', {}),
#     ('wv_ols', {}),
#     ('wv_ols_r2', {'emphasize_diversity': False}),
#     ('wv_ols_r2', {'emphasize_diversity': True}),
#     ('hard_voting', {'spread_window': 100, 'gaussian': False}),
#     ('hard_voting', {'spread_window': 100, 'gaussian': True})
# ]

# def create_heterogeneous_ensemble(n=ENSEMBLE_SIZE):
#     base_learners = []
#     selected_params = []
    
#     for base_learner_type in BASE_LEARNERS.keys():
#         base_learner_class = BASE_LEARNERS[base_learner_type]['class']
#         params_list = BASE_LEARNERS[base_learner_type]['params']
#         type_params = random.sample(params_list, BASE_LEARNERS_PER_TYPE)
        
#         for params in type_params:
#             base_learners.append(base_learner_class(**params))
#             selected_params.append((base_learner_type, params))
    
#     return base_learners, selected_params

# def run_experiment():
#     results = []
#     configurations = []
    
#     for rep in tqdm(range(N_REPETITIONS), desc="Running heterogeneous ensembles"):
#         base_learners, selected_params = create_heterogeneous_ensemble()
        
#         for method, method_params in ENSEMBLE_METHODS:
#             ensemble = EnsembleDetector(
#                 base_learners=base_learners,
#                 method=method,
#                 method_params=method_params,
#                 scores_dir=CACHED_SCORES_DIR
#             )
            
#             ensemble_name = ensemble.toString()
#             save_results_file = os.path.join(ENSEMBLE_RESULTS_DIR, f"{ensemble_name}_rep{rep}.csv")
#             save_scores_dir = os.path.join(ENSEMBLE_SCORES_DIR, f"{ensemble_name}_rep{rep}")
            
#             benchmark(ensemble, UCR_PATH, save_results_file, save_scores_dir)
            
#             benchmark_results = pd.read_csv(save_results_file, nrows=1)
            
#             result = {
#                 'ensemble_method': method,
#                 'method_params': json.dumps(method_params),  # Convert dict to string for storage in DataFrame
#                 'repetition': rep,
#                 'ucr_score': benchmark_results['accuracy'].values[0],
#                 'computational_time': benchmark_results['total_time'].values[0]
#             }
#             results.append(result)
            
#             configuration = {
#                 'ensemble_method': method,
#                 'method_params': method_params,
#                 'repetition': rep,
#                 'base_learner_params': selected_params
#             }
#             configurations.append(configuration)
    
#     return pd.DataFrame(results), configurations

# def analyze_results(results_df):
#     summary = results_df.groupby(['ensemble_method', 'method_params']).agg({
#         'ucr_score': ['mean', 'std', 'max'],
#         'computational_time': ['mean', 'std']
#     }).reset_index()
    
#     summary.columns = ['ensemble_method', 'method_params', 'mean_score', 'std_score', 'max_score', 'mean_time', 'std_time']
#     summary['cv_score'] = summary['std_score'] / summary['mean_score']
    
#     return summary



In [2]:
# Run the experiment
if not os.path.exists(ENSEMBLE_RESULTS_DIR):
    os.makedirs(ENSEMBLE_RESULTS_DIR)
if not os.path.exists(ENSEMBLE_SCORES_DIR):
    os.makedirs(ENSEMBLE_SCORES_DIR)

results_df, configurations = run_experiment()


Running heterogeneous ensembles:   0%|          | 0/30 [00:00<?, ?it/s]

CUDA is not available. Running on CPU.


Benchmarking: 100%|██████████| 250/250 [06:19<00:00,  1.52s/it]


Benchmark complete for ensemble_het_if_km_lof_mp_simple_average:
Total time: 5449.22 seconds
Accuracy: 55.20%
Correct predictions: 138
Incorrect predictions: 112
Failed predictions: 0
Results saved to: ensembles/results/ensemble_het_if_km_lof_mp_simple_average_rep0.csv
Scores saved to: ensembles/scores/ensemble_het_if_km_lof_mp_simple_average_rep0
CUDA is not available. Running on CPU.


In [ ]:
# results_df.to_csv('experiments/experiment_e3_results.csv', index=False)

# # Save configurations
# with open('experiments/experiment_e3_configurations.json', 'w') as f:
#     json.dump(configurations, f, indent=2)

# # Analyze the results
# summary = analyze_results(results_df)
# summary.to_csv('experiments/experiment_e3_summary.csv', index=False)

# print("Experiment E3 completed. Results saved to 'experiments/experiment_e3_results.csv', 'experiments/experiment_e3_configurations.json', and 'experiments/experiment_e3_summary.csv'.")

# # Load and print summary
# summary = pd.read_csv('experiments/experiment_e3_summary.csv')
# print(summary)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from matplotlib.colors import LinearSegmentedColormap

# Helper function to create unique method names
def get_unique_method_name(method, params):
    if method == 'wv_ols_r2':
        return f"wv_ols_r2_{str(params.get('emphasize_diversity', False))[0]}"
    elif method == 'hard_voting':
        return f"hard_voting_{str(params.get('gaussian', False))[0]}"
    elif params:
        param_str = '_'.join(f"{k}_{v}" for k, v in sorted(params.items()))
        return f"{method}_{param_str}"
    return method

# Load the data
e3_results = pd.read_csv('experiments/experiment_e3_results.csv')
e3_summary = pd.read_csv('experiments/experiment_e3_summary.csv')

# Load configurations
with open('experiments/experiment_e3_configurations.json', 'r') as f:
    e3_configs = json.load(f)

# Create unique method names
e3_results['unique_method'] = e3_results.apply(lambda row: get_unique_method_name(row['ensemble_method'], json.loads(row['method_params'])), axis=1)
e3_summary['unique_method'] = e3_summary.apply(lambda row: get_unique_method_name(row['ensemble_method'], json.loads(row['method_params'])), axis=1)

# Calculate relative performance for each ensemble
relative_performances = []
for config in e3_configs:
    method = config['ensemble_method']
    rep = config['repetition']
    constituent_scores = [e1_results[(e1_results['base_learner'] == bl_type) & 
                                     (e1_results['params'] == str(params))]['ucr_score'].values[0]
                          for bl_type, params in config['base_learner_params']]
    avg_constituent_score = np.mean(constituent_scores)
    unique_method = get_unique_method_name(method, config['method_params'])
    ensemble_data = e3_results[(e3_results['ensemble_method'] == method) & 
                               (e3_results['repetition'] == rep)]
    ensemble_score = ensemble_data['ucr_score'].values[0]
    relative_score = ensemble_score / avg_constituent_score
    relative_performances.append({
        'unique_method': unique_method,
        'repetition': rep,
        'relative_score': relative_score,
        'ensemble_score': ensemble_score,
        'avg_constituent_score': avg_constituent_score
    })

relative_df = pd.DataFrame(relative_performances)

# 1. Box Plot of UCR Scores
plt.figure(figsize=(15, 10))
sns.boxplot(x='unique_method', y='ucr_score', data=e3_results)
plt.title('Distribution of UCR Scores by Ensemble Method')
plt.xlabel('Ensemble Method')
plt.ylabel('UCR Score')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('visualisations/experiment_3/ucr_scores_boxplot.png')
plt.close()

# 2. Performance Improvement Violin Plot
plt.figure(figsize=(20, 12))
sns.violinplot(x='unique_method', y='relative_score', data=relative_df)
plt.axhline(y=1, color='r', linestyle='--')
plt.title('Ensemble Performance Relative to Constituent Learners')
plt.xlabel('Ensemble Method')
plt.ylabel('Relative Score')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('visualisations/experiment_3/relative_performance_violin.png')
plt.close()

# 3. Performance Improvement Bar Plot
plt.figure(figsize=(20, 12))
mean_relative_scores = relative_df.groupby('unique_method')['relative_score'].mean()
std_relative_scores = relative_df.groupby('unique_method')['relative_score'].std()

mean_relative_scores.plot(kind='bar', yerr=std_relative_scores, capsize=5)
plt.axhline(y=1, color='r', linestyle='--')
plt.title('Average Ensemble Performance Relative to Constituent Learners')
plt.xlabel('Ensemble Method')
plt.ylabel('Average Relative Score')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('visualisations/experiment_3/relative_performance_bar.png')
plt.close()

# 2. Performance Improvement Heatmap
avg_scores = e3_summary.groupby('unique_method')['mean_score'].mean()
improvement_pivot = pd.DataFrame({'Improvement': (avg_scores - avg_scores.mean()) / avg_scores.mean() * 100})

plt.figure(figsize=(12, 8))
colors = ['#FF0000', '#FFFFFF', '#00FF00']  # Red, White, Green
cmap = LinearSegmentedColormap.from_list('custom', colors, N=100)

sns.heatmap(improvement_pivot, annot=True, fmt='.2f', cmap=cmap, center=0)
plt.title('Performance Improvement of Ensemble Methods (%)')
plt.xlabel('Ensemble Method')
plt.ylabel('')
plt.tight_layout()
plt.savefig('visualisations/experiment_3/performance_improvement_heatmap.png')
plt.close()

# 3. Reliability Bar Chart
plt.figure(figsize=(15, 10))
cv_scores = e3_summary.set_index('unique_method')['cv_score']
cv_scores.plot(kind='bar')
plt.title('Coefficient of Variation of UCR Scores by Ensemble Method')
plt.xlabel('Ensemble Method')
plt.ylabel('Coefficient of Variation')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('visualisations/experiment_3/reliability_bar_chart.png')
plt.close()

# 4. Computational Time vs. Performance Scatter Plot
plt.figure(figsize=(15, 10))
sns.scatterplot(x='computational_time', y='ucr_score', hue='unique_method', data=e3_results)
plt.xlabel('Computational Time')
plt.ylabel('UCR Score')
plt.title('Performance vs Computational Time: Heterogeneous Ensemble Methods')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig('visualisations/experiment_3/performance_vs_time_scatter.png')
plt.close()

# 5. Performance Distribution Violin Plot
plt.figure(figsize=(15, 10))
sns.violinplot(x='unique_method', y='ucr_score', data=e3_results)
plt.title('Distribution of UCR Scores by Ensemble Method')
plt.xlabel('Ensemble Method')
plt.ylabel('UCR Score')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('visualisations/experiment_3/performance_distribution_violin.png')
plt.close()

print("All visualizations for Experiment 3 have been created and saved in the 'visualisations/experiment_3' directory.")

All visualizations for Experiment 3 have been created and saved in the 'visualisations/experiment_3' directory.
